In [1]:
from auxiliary_functions import generating_ADX_sample, preprocess_ADX_search_logs_insights_data
import re
import pandas as pd
import datetime

version = datetime.datetime.today().strftime('%d-%m-%Y')

# Setting up the initial parameters:

Below we display the only parameters you should be tweaking, which are the country code and the path where the functions should get the data from. This path is the path in the "/data" folder, where you should be storing the data from the responses taken from ADX.

Once this is done, you should only be pressing SHFT + ENTER until the end of the notebook. This notebook reads from the file selected, which should be in the parsed_data folder (where you have parsed the data from the search logs table in ADX).

This function will be making API calls to the TT API, so be mindful of the amount of data that you select as an input, as you may overwhelm the limits of the API. If you believe your process will take too long because of the volume of data, you should change the "sleep" parameter. The higher the value, the smaller the volume of requests you will be making to the API.

In [2]:
# Select the country code
country_code = 'US'

# Select reading path (should be located on the /data folder):
read_path = '/workspace/main_folder/main_folder/data/complete_responses_10-11-2022_US.parquet'

## Just execute...

In [3]:
# Advanced: You can change the save path here. If you have any doubts, just leave it as is
# This path creates itself automatically, so no need to change it.
save_path = f'/workspace/main_folder/main_folder/parsed_data/{country_code}_version{version}.parquet'

In [4]:
def extract_format_from_path(path: str) -> str:
    """Function that extracts the file from a path that contains the format at the end and no other points in the file name.
    
    :param path: Path where the data is stored. It will extract the format by getting all the letter after the only point in the path.
    :type path: str
    :return: The format in string format (for example 'csv' or 'parquet')
    :rtype: str
    """
    match = re.match('.+\.([\w]+)', path)
    
    if match is None:
        print('No path extracted')
    else:
        return match.group(1)
    
def parse_libpostal_and_structured_geocode(path: str, save_path: str, sep: str = ';', header: bool = True) -> pd.DataFrame:
    """Function that gets the DataFrame and parses the libpostal responses and the structured geocoding responses. It basically returns a DataFrame with all the relevant data parsed!
    
    :param df: DataFrame that contains the data as obtained from the sampling process in the search_logs_insights table in ADX.
    :type df: pd.DataFrame
    :param path: Specifies the path from which to read the data.
    :type path: str
    :param save_path: Specifies the path on which you want to save the data.
    :type save_path: str
    :param sep: Separator in case you use the csv format, defaults to ';'
    :type sep: str or None, optional
    :param header: Whether you want the first row of the csv to be the columns of the DataFrame, defaults to True, which means the first row of the csv has the column names for the DataFrame.
    :type header: bool, optional
    :return: A DataFrame with the parsed relevant responses
    :rtype: pd.DataFrame
    """
    file_format = extract_format_from_path(path)
    
    if file_format == 'csv':
        df = pd.read_csv(path, sep=sep, header=header)
    elif file_format == 'parquet':
        df = pd.read_parquet(path)
    else:
        raise TypeError('The format is not allowed of the file path is incorrect, check the file is csv or parquet, or that the file is stored in the location you passed')
        
    df = preprocess_ADX_search_logs_insights_data.parse_libpostal(df)
    df = preprocess_ADX_search_logs_insights_data.get_s2sG_results(df)

    display(df)
    
    df.to_parquet(save_path)

    return df

In [5]:
parse_libpostal_and_structured_geocode(
    path=read_path,
    save_path=save_path,
    sep=';',
    header=True
)

,request_uri,searched_query,populated_fields,countryName,who_searched,request_country,method_name,lib_postal_result,parsed_request_quertystring,developer_email,...,lp_po_box,lp_suburb,lp_city_district,lp_city,lp_island,lp_state_district,lp_state,lp_country_region,lp_country,lp_world_region
0,/geocode/2655%20mannheim%20rd.json?countrySet=...,2655 mannheim rd,4|5|,USA,NOT_SET,United States,search 2 geocode,"""{""""house_number"""":""""2655"""",""""road"""":""""mannhei...","""{""""countrySet"""":""""US%2CCA""""}""",christopher.witt@gm.com,...,,,,,,,,,,
1,/geocode/structured.xml?countryCode=US&postalC...,325539 US,11|19|,UNITED STATES OF AMERICA,NOT_SET,United States,search 2 structuredGeocode,,"""{""""countryCode"""":""""US"""",""""postalCode"""":""""3255...",marktj@wolfeinc.com,...,,,,,,,,,US,
2,/geocode/1555%20Blake%20Street%2080202.json?li...,1555 Blake Street 80202,4|5|11|,USA,NOT_SET,United States,search 2 geocode,"""{""""house_number"""":""""1555"""",""""road"""":""""blake s...","""{""""language"""":""""en-US"""",""""limit"""":""""10""""}""",VZ.NBI.VLT.Partner@verizon.com,...,,,,,,,,,,
3,/geocode/structured.json?countryCode=US&postal...,33810 US,11|19|,UNITED STATES OF AMERICA,NOT_SET,United States,search 2 structuredGeocode,,"""{""""limit"""":""""1"""",""""countryCode"""":""""US"""",""""pos...",Cesarm.863@gmail.com,...,,,,,,,,,US,
4,/geocode/structured.json?countryCode=US&street...,1300 RED JOHN DR DAYTONA BEACH FL US,5|14|19|,UNITED STATES OF AMERICA,NOT_SET,United States,search 2 structuredGeocode,,"""{""""countryCode"""":""""US"""",""""municipality"""":""""DA...",kmotwani@findersoftware.com,...,,,,DAYTONA%20BEACH,,,FL,,US,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,/geocode/structured.json?countryCode=US&street...,1617 SULTAN CIR 32766 CHULUOTA FL US,5|11|14|19|,UNITED STATES OF AMERICA,NOT_SET,United States,search 2 structuredGeocode,,"""{""""countryCode"""":""""US"""",""""municipality"""":""""CH...",kmotwani@findersoftware.com,...,,,,CHULUOTA,,,FL,,US,
49996,/geocode/structured.json?countryCode=US&street...,1243 GRASS FERN LN 32771 SANFORD FL US,5|11|14|19|,UNITED STATES OF AMERICA,NOT_SET,United States,search 2 structuredGeocode,,"""{""""countryCode"""":""""US"""",""""municipality"""":""""SA...",kmotwani@findersoftware.com,...,,,,SANFORD,,,FL,,US,
49997,/geocode/structured.json?countryCode=US&street...,12051 NW COPPER CREEK DR PORT ST LUCIE FL US,5|14|19|,UNITED STATES OF AMERICA,NOT_SET,United States,search 2 structuredGeocode,,"""{""""countryCode"""":""""US"""",""""municipality"""":""""PO...",kmotwani@findersoftware.com,...,,,,PORT%20ST%20LUCIE,,,FL,,US,
49998,/geocode/structured.json?countryCode=US&street...,S PENNSYLVANIA AVE 32789 WINTER PARK US,5|11|14|19|,UNITED STATES OF AMERICA,NOT_SET,United States,search 2 structuredGeocode,,"""{""""countryCode"""":""""US"""",""""postalCode"""":""""3278...",kmotwani@findersoftware.com,...,,,,WINTER%20PARK,,,,,US,


,request_uri,searched_query,populated_fields,countryName,who_searched,request_country,method_name,lib_postal_result,parsed_request_quertystring,developer_email,...,lp_po_box,lp_suburb,lp_city_district,lp_city,lp_island,lp_state_district,lp_state,lp_country_region,lp_country,lp_world_region
0,/geocode/2655%20mannheim%20rd.json?countrySet=...,2655 mannheim rd,4|5|,USA,NOT_SET,United States,search 2 geocode,"""{""""house_number"""":""""2655"""",""""road"""":""""mannhei...","""{""""countrySet"""":""""US%2CCA""""}""",christopher.witt@gm.com,...,,,,,,,,,,
1,/geocode/structured.xml?countryCode=US&postalC...,325539 US,11|19|,UNITED STATES OF AMERICA,NOT_SET,United States,search 2 structuredGeocode,,"""{""""countryCode"""":""""US"""",""""postalCode"""":""""3255...",marktj@wolfeinc.com,...,,,,,,,,,US,
2,/geocode/1555%20Blake%20Street%2080202.json?li...,1555 Blake Street 80202,4|5|11|,USA,NOT_SET,United States,search 2 geocode,"""{""""house_number"""":""""1555"""",""""road"""":""""blake s...","""{""""language"""":""""en-US"""",""""limit"""":""""10""""}""",VZ.NBI.VLT.Partner@verizon.com,...,,,,,,,,,,
3,/geocode/structured.json?countryCode=US&postal...,33810 US,11|19|,UNITED STATES OF AMERICA,NOT_SET,United States,search 2 structuredGeocode,,"""{""""limit"""":""""1"""",""""countryCode"""":""""US"""",""""pos...",Cesarm.863@gmail.com,...,,,,,,,,,US,
4,/geocode/structured.json?countryCode=US&street...,1300 RED JOHN DR DAYTONA BEACH FL US,5|14|19|,UNITED STATES OF AMERICA,NOT_SET,United States,search 2 structuredGeocode,,"""{""""countryCode"""":""""US"""",""""municipality"""":""""DA...",kmotwani@findersoftware.com,...,,,,DAYTONA%20BEACH,,,FL,,US,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,/geocode/structured.json?countryCode=US&street...,1617 SULTAN CIR 32766 CHULUOTA FL US,5|11|14|19|,UNITED STATES OF AMERICA,NOT_SET,United States,search 2 structuredGeocode,,"""{""""countryCode"""":""""US"""",""""municipality"""":""""CH...",kmotwani@findersoftware.com,...,,,,CHULUOTA,,,FL,,US,
49996,/geocode/structured.json?countryCode=US&street...,1243 GRASS FERN LN 32771 SANFORD FL US,5|11|14|19|,UNITED STATES OF AMERICA,NOT_SET,United States,search 2 structuredGeocode,,"""{""""countryCode"""":""""US"""",""""municipality"""":""""SA...",kmotwani@findersoftware.com,...,,,,SANFORD,,,FL,,US,
49997,/geocode/structured.json?countryCode=US&street...,12051 NW COPPER CREEK DR PORT ST LUCIE FL US,5|14|19|,UNITED STATES OF AMERICA,NOT_SET,United States,search 2 structuredGeocode,,"""{""""countryCode"""":""""US"""",""""municipality"""":""""PO...",kmotwani@findersoftware.com,...,,,,PORT%20ST%20LUCIE,,,FL,,US,
49998,/geocode/structured.json?countryCode=US&street...,S PENNSYLVANIA AVE 32789 WINTER PARK US,5|11|14|19|,UNITED STATES OF AMERICA,NOT_SET,United States,search 2 structuredGeocode,,"""{""""countryCode"""":""""US"""",""""postalCode"""":""""3278...",kmotwani@findersoftware.com,...,,,,WINTER%20PARK,,,,,US,


In [6]:
test = pd.read_parquet(save_path)
test

,request_uri,searched_query,populated_fields,countryName,who_searched,request_country,method_name,lib_postal_result,parsed_request_quertystring,developer_email,...,lp_po_box,lp_suburb,lp_city_district,lp_city,lp_island,lp_state_district,lp_state,lp_country_region,lp_country,lp_world_region
0,/geocode/2655%20mannheim%20rd.json?countrySet=...,2655 mannheim rd,4|5|,USA,NOT_SET,United States,search 2 geocode,"""{""""house_number"""":""""2655"""",""""road"""":""""mannhei...","""{""""countrySet"""":""""US%2CCA""""}""",christopher.witt@gm.com,...,,,,,,,,,,
1,/geocode/structured.xml?countryCode=US&postalC...,structuredGeocode.xml,11|19|,UNITED STATES OF AMERICA,NOT_SET,United States,search 2 structuredGeocode,,"""{""""countryCode"""":""""US"""",""""postalCode"""":""""3255...",marktj@wolfeinc.com,...,,,,,,,,,US,
2,/geocode/1555%20Blake%20Street%2080202.json?li...,1555 Blake Street 80202,4|5|11|,USA,NOT_SET,United States,search 2 geocode,"""{""""house_number"""":""""1555"""",""""road"""":""""blake s...","""{""""language"""":""""en-US"""",""""limit"""":""""10""""}""",VZ.NBI.VLT.Partner@verizon.com,...,,,,,,,,,,
3,/geocode/structured.json?countryCode=US&postal...,structuredGeocode.json,11|19|,UNITED STATES OF AMERICA,NOT_SET,United States,search 2 structuredGeocode,,"""{""""limit"""":""""1"""",""""countryCode"""":""""US"""",""""pos...",Cesarm.863@gmail.com,...,,,,,,,,,US,
4,/geocode/structured.json?countryCode=US&street...,structuredGeocode.json,5|14|19|,UNITED STATES OF AMERICA,NOT_SET,United States,search 2 structuredGeocode,,"""{""""countryCode"""":""""US"""",""""municipality"""":""""DA...",kmotwani@findersoftware.com,...,,,,DAYTONA%20BEACH,,,FL,,US,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,/geocode/structured.json?countryCode=US&street...,structuredGeocode.json,5|11|14|19|,UNITED STATES OF AMERICA,NOT_SET,United States,search 2 structuredGeocode,,"""{""""countryCode"""":""""US"""",""""municipality"""":""""CH...",kmotwani@findersoftware.com,...,,,,CHULUOTA,,,FL,,US,
49996,/geocode/structured.json?countryCode=US&street...,structuredGeocode.json,5|11|14|19|,UNITED STATES OF AMERICA,NOT_SET,United States,search 2 structuredGeocode,,"""{""""countryCode"""":""""US"""",""""municipality"""":""""SA...",kmotwani@findersoftware.com,...,,,,SANFORD,,,FL,,US,
49997,/geocode/structured.json?countryCode=US&street...,structuredGeocode.json,5|14|19|,UNITED STATES OF AMERICA,NOT_SET,United States,search 2 structuredGeocode,,"""{""""countryCode"""":""""US"""",""""municipality"""":""""PO...",kmotwani@findersoftware.com,...,,,,PORT%20ST%20LUCIE,,,FL,,US,
49998,/geocode/structured.json?countryCode=US&street...,structuredGeocode.json,5|11|14|19|,UNITED STATES OF AMERICA,NOT_SET,United States,search 2 structuredGeocode,,"""{""""countryCode"""":""""US"""",""""postalCode"""":""""3278...",kmotwani@findersoftware.com,...,,,,WINTER%20PARK,,,,,US,
